In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/home/ethereum/Downloads/ResearchData/data/order10k.csv")

In [3]:
df.iloc[0]

o_orderkey               1
o_custkey              259
o_totalprice     153389.02
o_orderdate     1996-01-02
Name: 0, dtype: object